## `fit_residuals.ipynb`
Notebook by Tim Bartholomaus, June 18, 2019

Goal of notebook is to plot residuals between a reference profile/surface as a function of space and time, and then fit a smooth, interpolated surface through these residuals.

Residuals are expected in a dictionary, `res` with keys `"x"`, `"t"`, and `"off"`, in which `x` is position along the profile, `t` is the relative time of the residual measurement, and `off` is the measured offset between the ICESat-2 measurement and the reference surface elevation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy import interpolate

import sys
sys.path.insert(0, '~/gridding/notebook')
# import utils

# print(sys.path)

In [2]:
print(sys.path)

['~/gridding/notebook', '/srv/conda/lib/python36.zip', '/srv/conda/lib/python3.6', '/srv/conda/lib/python3.6/lib-dynload', '', '/srv/conda/lib/python3.6/site-packages', '/srv/conda/lib/python3.6/site-packages/IPython/extensions', '/home/jovyan/.ipython']


#### Create synthetic residuals that are a function of length along the profile `x`, and time `t`

In [ ]:
# n_residuals = 30
# max_length = 50*1000 # m
# max_time = 100 # d
# max_off = 100 # m


# res = dict()
# res['x'] = max_length * np.random.rand(n_residuals) # m
# res['t'] = max_time * np.random.rand(n_residuals) # d

# noise = np.random.randn(n_residuals) * 20 # m

# off_scalar = max_off / (max_length*max_time)
# res['off'] = off_scalar * res['x']*res['t'] + noise # create the synthetic residuals, with noise

#### Read in measured residuals that are a function of length along the profile `x`, and time `t`

In [18]:
atm_date = '2014-04-29'
# atm_date = '2018-04-18'

data_prod_path = '../data_prod/'
# intersections_filename = 'Intersections_ATM20180418.csv'
# intersections_filename = 'Intersections_ATM'+atm_date[0:4] + atm_date[5:7] + atm_date[8:10] + '.csv'
intersections_filename = 'residuals.csv'
res = pd.read_csv(data_prod_path + intersections_filename, parse_dates=[5]) 

res.head()

res.dtypes

res['off'] = res['residuals']
# res['off'] = res['z_ATL06'] - res['ATM_elev']
res['t'] = res['t_ATL06'] - pd.to_datetime(atm_date)
res['t_days'] = res['t'].astype('timedelta64[D]')
res['t_flt'] = res['t_days'].astype('float64')
res.rename(columns={'dist_along': "x"}, inplace=True)


# res.head()
# # res.columns

# pd.to_datetime(atm_date)

In [ ]:
res['t_ATL06'] - pd.to_datetime(atm_date)

In [ ]:

# n_residuals = 30
# max_length = 50*1000 # m
# max_time = 100 # d
# max_off = 100 # m


# res = dict()
# res['x'] = max_length * np.random.rand(n_residuals) # m
# res['t'] = max_time * np.random.rand(n_residuals) # d

# noise = np.random.randn(n_residuals) * 20 # m

# off_scalar = max_off / (max_length*max_time)
# res['off'] = off_scalar * res['x']*res['t'] + noise # create the synthetic residuals, with noise

#### Spatio-temporal (2-D) interpolation
Right now, the radial basis functions seem to perform best

In [17]:
res['t']

0     1633 days 15:53:52
1     1633 days 15:53:52
2     1633 days 15:53:52
3     1633 days 15:53:52
4     1636 days 05:21:45
5     1640 days 05:13:27
6     1640 days 05:13:27
7     1640 days 05:13:27
8     1640 days 05:13:27
9     1640 days 05:13:27
10    1641 days 15:37:16
11    1641 days 15:37:16
12    1641 days 15:37:16
13    1641 days 15:37:16
14    1641 days 15:37:16
15    1641 days 15:37:16
16    1657 days 04:14:44
17    1657 days 04:14:44
18    1657 days 04:14:44
19    1657 days 04:14:44
20    1657 days 04:14:44
21    1657 days 04:14:44
22    1661 days 04:06:22
23    1661 days 04:06:22
24    1661 days 04:06:22
25    1661 days 04:06:22
26    1661 days 04:06:22
27    1661 days 04:06:22
28    1665 days 03:57:57
29    1665 days 03:57:57
             ...        
153   1747 days 23:54:28
154   1747 days 23:54:28
155   1747 days 23:54:28
156   1747 days 23:54:28
157   1751 days 23:46:06
158   1751 days 23:46:06
159   1751 days 23:46:06
160   1751 days 23:46:06
161   1751 days 23:46:06


In [39]:
res['x']


0      122581.170689
1      122647.058967
2      125810.129706
3      125875.304455
4       77393.293766
5       67655.902876
6       67687.240097
7       64484.432996
8       61379.685494
9       61315.684441
10      89377.265064
11      89479.464606
12      92780.627483
13      92848.522698
14      96176.153026
15      96241.609114
16     121954.028971
17     121854.851720
18     118653.494370
19     118588.415511
20     115361.066547
21     115295.191381
22     105596.029174
23     105001.339697
24     102145.248209
25     102080.404536
26      98960.114430
27      98894.603155
28      89241.058144
29      89172.935318
           ...      
153    121425.520431
154    121359.531822
155    118164.360812
156    118098.943857
157    108153.357245
158    108088.013034
159    104835.607040
160    104769.182503
161    101658.707466
162    101593.796438
163    125289.795232
164    125354.749926
165     91929.771598
166     91861.692774
167     88729.738750
168     88661.510375
169     85556

In [103]:
# # Create a mesh of the space and time variables
# grid_x, grid_t = np.mgrid[0:max_length:100j, 
#                           0:max_time:100j] # create a grid from 0 to max_length and 0 to
#                                            #  max_time, with the j number of steps
# Create a mesh of the space and time variables
grid_x, grid_t = np.mgrid[50000:130000:100j, 
                          1620:1780:100j] # create a grid from 0 to max_length and 0 to
                                           #  max_time, with the j number of steps
    
# # # This griddata is too strict
grid_off = interpolate.griddata((res['x'].values, res['t_flt'].values), res['off'].values, 
                                (grid_x, grid_t), method='linear')

# tck = interpolate.bisplrep(res['x'].values, res['t'].values, res['off'].values, s=2)
# grid_off = interpolate.bisplev(grid_x[:,0], grid_t[0,:], tck)

# f = interpolate.interp2d(res['x'].values, res['t_flt'].values, res['off'].values, kind='linear')
# grid_off = f(grid_x[:,0], grid_t[0,:])

# # Radial basis functions
# x_stan = res['x']/150#max_length
# t_stan = res['t']/1800#max_time
# rbf = interpolate.Rbf(x_stan, t_stan, res['off'], epsilon=1)
#         # epsilon defines the stiffness of the fitting routine. High values are stiff surfaces
# grid_x_stan = grid_x/150#max_length
# grid_t_stan = grid_t/1800#max_time
# grid_off = rbf(grid_x_stan, grid_t_stan)


# # Kriging
# # Function does not like NaN's, so lets remove them
# # xp, yp, zp = x[~np.isnan(z)], y[~np.isnan(z)], z[~np.isnan(z)]
# Run kriging interpolator, with 1 m RMSE noise added to the diagonal (np.ones(xp.shape)*1)
# grid_off_lsc, e_lsc = utils.lscip(res['x'], res['t'], res['off'], np.ones(res['x'].shape)*1, grid_x, grid_t, n=10, d=1e3, a=25e3)[0:2]

In [98]:
grid_off

array([[-6.55931481e-02, -6.55931481e-02, -6.55931481e-02, ...,
        -3.06234381e+00, -3.06234381e+00, -3.06234381e+00],
       [-6.55931481e-02, -6.55931481e-02, -6.55931481e-02, ...,
        -3.06234381e+00, -3.06234381e+00, -3.06234381e+00],
       [-6.55931481e-02, -6.55931481e-02, -6.55931481e-02, ...,
        -3.06234381e+00, -3.06234381e+00, -3.06234381e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.26824834e+01, -9.26824834e+01, -9.26824834e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.26824834e+01, -9.26824834e+01, -9.26824834e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.26824834e+01, -9.26824834e+01, -9.26824834e+01]])

#### Plotting

In [25]:
%matplotlib widget
plt.figure(figsize=(10,6))
# plt.contourf(grid_x/1000, grid_t, (grid_off))
plt.scatter(res['x'].values/1000, res['t_days'].values, s = 20, c=res['off'].values,
            linewidth=.5, edgecolor='k', vmin = -50, vmax=10, cmap='inferno')
# res.plot.scatter(x='x', y='t', c='off')#, linewidth=.5, edgecolor='k')
plt.xlabel('Profile Length (km)')
plt.ylabel('Time (yrs)')
# plt.colormap('RdBu')
plt.colorbar(label='Deviation from reference profile (m)');
plt.show()

FigureCanvasNbAgg()

In [80]:
grid_x

array([[ 50.        ,  50.        ,  50.        , ...,  50.        ,
         50.        ,  50.        ],
       [ 50.80808081,  50.80808081,  50.80808081, ...,  50.80808081,
         50.80808081,  50.80808081],
       [ 51.61616162,  51.61616162,  51.61616162, ...,  51.61616162,
         51.61616162,  51.61616162],
       ...,
       [128.38383838, 128.38383838, 128.38383838, ..., 128.38383838,
        128.38383838, 128.38383838],
       [129.19191919, 129.19191919, 129.19191919, ..., 129.19191919,
        129.19191919, 129.19191919],
       [130.        , 130.        , 130.        , ..., 130.        ,
        130.        , 130.        ]])

In [104]:
# fig, ax = plt.subplots(3)

plt.figure()
plt.contourf(grid_x/1000, grid_t, grid_off, cmap='inferno')
# plt.scatter(res['x']/1000, res['t'], c=res['off'], linewidth=.5, edgecolor='k')
plt.scatter(res['x'].values/1000, res['t_days'].values, s = 20, c=res['off'].values,
            linewidth=.5, edgecolor='k', vmin = -50, vmax=10, cmap='inferno')
plt.xlabel('Profile Length (km)')
plt.ylabel('Time (days)')
plt.colorbar(label='Deviation from reference profile (m)');
plt.show()

FigureCanvasNbAgg()

In [96]:
print(grid_t, res['t_days'].values)

[[1620.         1621.61616162 1623.23232323 ... 1776.76767677
  1778.38383838 1780.        ]
 [1620.         1621.61616162 1623.23232323 ... 1776.76767677
  1778.38383838 1780.        ]
 [1620.         1621.61616162 1623.23232323 ... 1776.76767677
  1778.38383838 1780.        ]
 ...
 [1620.         1621.61616162 1623.23232323 ... 1776.76767677
  1778.38383838 1780.        ]
 [1620.         1621.61616162 1623.23232323 ... 1776.76767677
  1778.38383838 1780.        ]
 [1620.         1621.61616162 1623.23232323 ... 1776.76767677
  1778.38383838 1780.        ]] [1633. 1633. 1633. 1633. 1636. 1640. 1640. 1640. 1640. 1640. 1641. 1641.
 1641. 1641. 1641. 1641. 1657. 1657. 1657. 1657. 1657. 1657. 1661. 1661.
 1661. 1661. 1661. 1661. 1665. 1665. 1665. 1665. 1665. 1665. 1666. 1666.
 1666. 1666. 1666. 1666. 1669. 1669. 1669. 1669. 1669. 1669. 1670. 1670.
 1670. 1670. 1670. 1670. 1674. 1674. 1674. 1674. 1674. 1674. 1686. 1686.
 1686. 1686. 1686. 1686. 1690. 1690. 1690. 1690. 1690. 1690. 1694. 1694

In [109]:
grid_t[:,50]
np.savetxt('residuals_at_1700d.csv', grid_off[:,50]) 
np.savetxt('along_dist_at_1700d.csv', grid_x[:,50]) 
